# BERT（Keras BERT）を使用した文章分類を学習から予測まで紹介！ | cloud.config Tech Blog
https://tech-blog.cloud-config.jp/2020-02-06-category-classification-using-bert

https://yoheikikuta.github.io/bert-japanese/  
必要データ一覧

BERT用のファイル
*   model.ckpt-1400000.data-00000-of-00001
*   model.ckpt-1400000.meta
*   model.ckpt-1400000.index

SentencePiece用のファイル
*   wiki-ja.vocab
*   wiki-ja.model

## ライブラリのインストール

In [ ]:
!pip install sentencepiece
!pip install keras_bert

     |████████████████████████████████| 1.1MB 6.7MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=571eda3e0b4b74a0cb71955168fd7a133abe4d7eb7209db2b1aaae75c879bab0
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=e08a7ce95054574cbcfb1c9ec6e4f96f4eb9683c35152d2ace18d882fdd1547d
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=dc5c63784985c1e234869337382d538a3d893b6f76c87859ffa26d6ed320b13d
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=0d6834e3479a5314

In [ ]:
import pandas as pd
import numpy as np
import sentencepiece as spm
import csv
from google.colab import drive
from google.colab import files
import sys
from keras_bert import load_trained_model_from_checkpoint

In [ ]:
!nvidia-smi

Fri Nov  6 04:16:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## GoogleDriveのマウント

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/My Drive/dev/bertclass/'

## データ準備

In [ ]:
# CSVとして保存
# Pythonでcsvファイルへの書き出し(list,numpy,pandas対応) | 超初心者向けPython入門講座
# https://work-life-enj.com/pyhon-write-csv/

f = open(data_dir + 'KNBC/all.csv')
data = csv.reader(f)

test_data = []

for row in data:
  test_data.append(row)

f.close()

print(test_data[:3])

# feature/labelを列ごとに取得
# features = [item[1] for item in test_data]
# 1個のデータをリストとして登録しないと、CSVに落とした時に、1文字1要素に分解される
features,labels = [],[]
for item in test_data:
    if item[1] == "":
        continue
    features.append([item[1]])
    labels.append([item[0]])
print(features[:4])
print(labels[:4])


[['Gourmet', '烏丸六角のおかき屋さん'], ['Gourmet', '六角堂の前にある、蕪村庵というお店に行ってきた。'], ['Gourmet', 'おかきやせんべいの店なのだが、これがオイシイ。']]
[['烏丸六角のおかき屋さん'], ['六角堂の前にある、蕪村庵というお店に行ってきた。'], ['おかきやせんべいの店なのだが、これがオイシイ。'], ['のれんをくぐると小さな庭があり、その先に町屋風の店内がある。']]
[['Gourmet'], ['Gourmet'], ['Gourmet'], ['Gourmet']]


In [ ]:
from sklearn.model_selection import train_test_split

# ランダムシード：0、データの並び替え：する
x_train, x_test, t_train, t_test = train_test_split(features, labels, train_size=0.7, random_state=0)

def list2csv_dl(list,title,csvfile):
  f = open(csvfile, 'w')
  # 必ずダブルクオーテーションで囲むように
  writer = csv.writer(f, quoting=csv.QUOTE_ALL)
  writer.writerow([title])
  writer.writerows(list)
  f.close()
  files.download(csvfile)
  print(list[:3])

list2csv_dl(x_train,'feature','feature_train.csv')
list2csv_dl(x_test,'feature','feature_test.csv')
list2csv_dl(t_train,'label','label_train.csv')
list2csv_dl(t_test,'label','label_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[['・・・とうとう見てしまいましたよ。'], ['・パソコンのかわりにはなりません'], ['食べるのは大好き！！']]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

[['Keitai'], ['Keitai'], ['Gourmet']]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[['Kyoto'], ['Keitai'], ['Keitai']]


### 最大トークン数の調査

max_token_number: 3138

max_position_embeddingsとmax_seq_lengthの値を使用する学習データセットの最大トークン数に
bert_finetuning_config_v1.json

In [ ]:
# feature.csvは上記で用意したファイルのパスを指定してください
train_features_df = pd.read_csv(data_dir + 'trains/feature_train.csv')
print(train_features_df.head())
def _get_indice(feature):
    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')
    number = len(tokens)

    return number

sp = spm.SentencePieceProcessor()
# ダウンロードした事前学習モデルのパスを指定してください
sp.Load(data_dir + 'wiki-ja.model')

numbers = []

for feature in train_features_df['feature']:
    features_number = _get_indice(feature)
    numbers.append(features_number)

# 最大トークン数
max_token_num = max(numbers)
print("max_token_number: " + str(max_token_num))

                                             feature
0                                  ・・・とうとう見てしまいましたよ。
1                                   ・パソコンのかわりにはなりません
2                                         食べるのは大好き！！
3                                               ・鹿苑寺
4  『あぁー……。まぁ買う人もいるんですけどねー……（笑）…ぁ、チョクチョクいるんですよ（笑）ホ...
max_token_number: 103


## BERTの設定ファイル、モデルのロード

学習回数と事前に調べていた最大トークン数、ファイルパスを自分用に書き換えてください。以下に書き換える箇所を示します。


*   config_path：設定ファイルのパス
*   checkpoint_path：事前学習モデルのファイルパス
    *   拡張子まで書かないでください
*   SEQ_LEN：最大トークン数
*   EPOCH：学習回数

```
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py in set_weights(self, weights)
   1824           raise ValueError(
   1825               'Layer weight shape %s not compatible with provided weight '
-> 1826               'shape %s' % (ref_shape, weight.shape))
   1827         weight_value_tuples.append((param, weight))
   1828         weight_index += 1

ValueError: Layer weight shape (3138, 768) not compatible with provided weight shape (512, 768)
```

In [ ]:
%%time
sys.path.append('modules')

# BERTのロード
config_path = data_dir + 'bert_finetuning_config_v1.json'
# 拡張子まで記載しない
checkpoint_path = data_dir + 'model.ckpt-1400000'

# 最大のトークン数
SEQ_LEN = 103
BATCH_SIZE = 16
BERT_DIM = 768
LR = 1e-4
# 学習回数
EPOCH = 20

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True,  trainable=True, seq_len=SEQ_LEN)
bert.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 103)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 103)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 103, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 103, 768)     1536        Input-Segment[0][0]              
_______________________________________________________________________________________

In [ ]:
from keras import utils


maxlen = SEQ_LEN
 
sp = spm.SentencePieceProcessor()
sp.Load(data_dir + 'wiki-ja.model')

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype = np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = sp.piece_to_id(token)
        except:
            logging.warn(f'{token} is unknown.')
            indices[t] = sp.piece_to_id('<unk>')

    return indices

def _load_labeldata(train_dir, test_dir):
    train_features_df = pd.read_csv(f'{train_dir}/feature_train.csv')
    train_labels_df = pd.read_csv(f'{train_dir}/label_train.csv')
    test_features_df = pd.read_csv(f'{test_dir}/feature_test.csv')
    test_labels_df = pd.read_csv(f'{test_dir}/label_test.csv')
    label2index = {k: i for i, k in enumerate(train_labels_df['label'].unique())}
    index2label = {i: k for i, k in enumerate(train_labels_df['label'].unique())}
    class_count = len(label2index)
    # train_labels = utils.np_utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    train_labels = utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    test_label_indices = [label2index[label] for label in test_labels_df['label']]
    # test_labels = utils.np_utils.to_categorical(test_label_indices, num_classes=class_count)
    test_labels = utils.to_categorical(test_label_indices, num_classes=class_count)

    train_features = []
    test_features = []

    for feature in train_features_df['feature']:
        train_features.append(_get_indice(feature))
    train_segments = np.zeros((len(train_features), maxlen), dtype = np.float32)
    for feature in test_features_df['feature']:
        test_features.append(_get_indice(feature))
    test_segments = np.zeros((len(test_features), maxlen), dtype = np.float32)

    print(f'Trainデータ数: {len(train_features_df)}, Testデータ数: {len(test_features_df)}, ラベル数: {class_count}')

    return {
        'class_count': class_count,
        'label2index': label2index,
        'index2label': index2label,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'test_label_indices': test_label_indices,
        'train_features': np.array(train_features),
        'train_segments': np.array(train_segments),
        'test_features': np.array(test_features),
        'test_segments': np.array(test_segments),
        'input_len': maxlen
    }

## モデル作成関数

In [ ]:
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten, GlobalMaxPooling1D
from keras_bert.layers import MaskedGlobalMaxPool1D
from keras import Input, Model
from keras_bert import AdamWarmup, calc_train_steps

def _create_model(input_shape, class_count):
    decay_steps, warmup_steps = calc_train_steps(
        input_shape[0],
        batch_size=BATCH_SIZE,
        epochs=EPOCH,
    )

    bert_last = bert.get_layer(name='NSP-Dense').output
    x1 = bert_last
    output_tensor = Dense(class_count, activation='softmax')(x1)
    # Trainableの場合は、Input Masked Layerが3番目の入力なりますが、
    # FineTuning時には必要無いので1, 2番目の入力だけ使用します。
    # Trainableでなければkeras-bertのModel.inputそのままで問題ありません。
    model = Model([bert.input[0], bert.input[1]], output_tensor)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
                  #optimizer='nadam',
                  metrics=['mae', 'mse', 'acc'])

    return model

## 学習データのロードとモデルの準備
事前準備で作成した学習用データと学習後のモデル名および出力先を指定してください。  

*   trains_dir,tests_dir：学習用データのパス
*   model_filename：学習後のモデル名、出力先のパス

In [ ]:
%%time
# データロードとモデルの準備
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

trains_dir = data_dir + 'trains'
tests_dir = data_dir + 'tests'

data = _load_labeldata(trains_dir, tests_dir)
model_filename = data_dir + 'models/ldnews_finetuning.model'
model = _create_model(data['train_features'].shape, data['class_count'])

model.summary()

Trainデータ数: 2919, Testデータ数: 1252, ラベル数: 4
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 103)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 103)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 103, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 103, 768)     1536        Input-Segment[0][0]              
______________________________________________

## 計算途中のモデルの読み込み

In [ ]:
model.load_weights(model_filename)
# loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100*acc))

OSError: ignored

## 学習の実行

In [ ]:
%%time
history = model.fit([data['train_features'], data['train_segments']],
          data['train_labels'],
          epochs = EPOCH-17,
          batch_size = BATCH_SIZE,
          validation_data=([data['test_features'], data['test_segments']], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])

Epoch 1/3


ValueError: ignored

In [ ]:
# モデルの保存と復元  |  TensorFlow Core
# https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja

'''
model_path  = './'

model_flist = os.listdir(model_path)
model_flist = glob.glob(model_path + '*.pkl');

// Modelのload
models = {}
for model_file in model_flist:
    model_name = model_file[:-4]
    models[model_name] = pickle.load(open(model_path + model_file, 'rb'))

// Modelのsave
for model_name in models.keys():
    pickle.dump(models[model_name], open(model_path + model_name + '.pkl', 'wb'))
'''

TPU：1EPOCHあたり50～60分
GPI(T4)：1EPOCHあたり3分弱！


## モデルの評価

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
from keras_bert import get_custom_objects

In [ ]:
%%time
model = load_model(model_filename, custom_objects=get_custom_objects())

predicted_test_labels = model.predict([data['test_features'], data['test_segments']]).argmax(axis=1)
numeric_test_labels = np.array(data['test_labels']).argmax(axis=1)

report = classification_report(
        numeric_test_labels, predicted_test_labels, target_names=['Gourmet', 'Keitai', 'Kyoto', 'Sports'], output_dict=True)

display(pd.DataFrame(report).T)

,precision,recall,f1-score,support
Gourmet,0.815934,0.787798,0.801619,377.000000
Keitai,0.767442,0.717391,0.741573,276.000000
Kyoto,0.787942,0.865297,0.824810,438.000000
Sports,0.798658,0.739130,0.767742,161.000000
accuracy,0.793131,0.793131,0.793131,0.793131
macro avg,0.792494,0.777404,0.783936,1252.000000
weighted avg,0.793230,0.793131,0.792139,1252.000000


CPU times: user 19.9 s, sys: 3.39 s, total: 23.3 s
Wall time: 34.7 s


9回回した後の結果

data |precision|recall|f1-score|support
--- | --- | --- | --- | ---
Gourmet|0.832869|0.793103|0.812500|377.000000
Keitai|0.663580|0.778986|0.716667|276.000000
Kyoto|0.801418|0.773973|0.787456|438.000000
Sports|0.767123|0.695652|0.729642|161.000000
accuracy|0.770767|0.770767|0.770767|0.770767
macro avg|0.766248|0.760428|0.761566|1252.000000
weighted avg|0.776093|0.770767|0.771957|1252.000000

### モデルの学習再開

In [ ]:
%%time
model = load_model(model_filename, custom_objects=get_custom_objects())

CPU times: user 15.5 s, sys: 3.06 s, total: 18.5 s
Wall time: 21.5 s


In [ ]:
history = model.fit([data['train_features'], data['train_segments']],
          data['train_labels'],
          epochs = EPOCH-10,
          batch_size = BATCH_SIZE,
          validation_data=([data['test_features'], data['test_segments']], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])

Epoch 1/10
183/183 [==============================] - 159s 867ms/step - loss: 0.0234 - mean_absolute_error: 0.0035 - mean_squared_error: 0.0025 - acc: 0.9935 - val_loss: 1.4834 - val_mean_absolute_error: 0.1152 - val_mean_squared_error: 0.1010 - val_acc: 0.7756
Epoch 2/10
183/183 [==============================] - 131s 718ms/step - loss: 0.0605 - mean_absolute_error: 0.0114 - mean_squared_error: 0.0066 - acc: 0.9832 - val_loss: 1.4211 - val_mean_absolute_error: 0.1191 - val_mean_squared_error: 0.1052 - val_acc: 0.7652
Epoch 3/10
183/183 [==============================] - 125s 684ms/step - loss: 0.0843 - mean_absolute_error: 0.0164 - mean_squared_error: 0.0096 - acc: 0.9767 - val_loss: 1.1512 - val_mean_absolute_error: 0.1339 - val_mean_squared_error: 0.1078 - val_acc: 0.7428
Epoch 4/10
183/183 [==============================] - 125s 681ms/step - loss: 0.0653 - mean_absolute_error: 0.0128 - mean_squared_error: 0.0071 - acc: 0.9822 - val_loss: 1.3176 - val_mean_absolute_error: 0.1388 - v

## 予測

### 文字列から予測(出力は数字)

In [ ]:
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix


sys.path.append('modules')

# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load(data_dir + 'wiki-ja.model')
# BERTの学習したモデルの読込
model_filename = data_dir + 'models/ldnews_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())

SEQ_LEN = 103
maxlen = SEQ_LEN

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
    return indices

feature = "昨日は携帯電話を買いに行った。"

test_features = []
test_features.append(_get_indice(feature))
test_segments = np.zeros(
    (len(test_features), maxlen), dtype=np.float32)

predicted_test_labels = model.predict(
    [test_features, test_segments]).argmax(axis=1)

print(predicted_test_labels[0])

ValueError: ignored

### ラベル番号をラベル名に変換するように

In [ ]:
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix


sys.path.append('modules')

# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load(data_dir + 'wiki-ja.model')
# BERTの学習したモデルの読込
model_filename = data_dir + 'models/ldnews_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())

SEQ_LEN = 103
maxlen = SEQ_LEN

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
    return indices

feature = "昨日は携帯電話を買いに行った。"

test_features = []
test_features.append(_get_indice(feature))
test_segments = np.zeros(
    (len(test_features), maxlen), dtype=np.float32)

predicted_test_labels = model.predict(
    [test_features, test_segments]).argmax(axis=1)

label_data = pd.read_csv(data_dir + 'label_id/id_category.csv')
label = label_data.query(f'id == {predicted_test_labels[0]}')
label = label.iloc[0]
label_name = label['label']
print(label_name)

ValueError: ignored